In [19]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.globals import set_debug

# Variables

In [20]:
load_dotenv(override=True)

True

In [21]:
file = os.getenv("FILE")
persist_directory = os.getenv("PERSIST_DIRECTORY") + "/chroma"
api_key = os.getenv("OPENAI_API_KEY")

# PDFLoader

In [22]:
loader = PyPDFLoader(file)
files = loader.load()

In [23]:
id = "91513024-a6d7-4404-8fc7-a3b51bf38347"

In [24]:
data = []

for file in files:
    file.metadata['id'] = str(id)
    
data.extend(files)

# Recursive TextSplit

In [25]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

In [26]:
docs = recur_split.split_documents(data)

# Embbiding (OpenAI)

In [27]:
embeddings = OpenAIEmbeddings()

# Vector Store

In [28]:
%run mongo.ipynb

In [29]:
connection_manager = MongoDBConnectionManager()
handler = MongoDBDocumentHandler(embeddings, connection_manager)

In [30]:
handler.add_documents(docs)

In [31]:

retriever = handler.get_retriever(
    search_type="mmr",
    search_kwargs={
        'k': 3,
        'fetch_k': 4,
        'pre_filter': {'id': str(id)}
    }
)

# LLM (OpenAI)

In [32]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

# Chain

In [33]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
)

# Question

In [34]:
question = "O que é EF09MA01?"

# Answer

In [35]:
chat_chain.invoke({"query": question})

{'query': 'O que é EF09MA01?',
 'result': 'EF09MA01 é uma referência a um componente da Base Nacional Comum Curricular (BNCC) do Brasil, especificamente relacionado ao ensino de Matemática para o 9º ano do Ensino Fundamental. Essa referência indica um conjunto de habilidades e competências que os alunos devem desenvolver nesse nível de ensino. Para informações mais detalhadas sobre o que exatamente abrange essa habilidade, seria necessário consultar a BNCC ou documentos relacionados à educação brasileira.'}

# Prompt

In [36]:
prompt = PromptTemplate.from_template(
"""
Você é um assistente educacional que retorna codigos BNCC e suas descricoes.

O usuário fornecerá os dados de uma aula em formato estruturado, incluindo:
**Ano Letivo**;
**Disciplina**;
**Conteúdo**;
**Objetivo da Aula**.

Com base no documento (que está no contexto abaixo), identifique **um ou mais BNCCs e descricoes** que se relacionam diretamente com essa aula.
Traga somente o(s) BNCC(s) que estão no documento abaixo, referente ao Ano Letivo, a Disiciplina, o Conteudo e Objetivo da Aula.
Retorne apenas os codigos e as descricoes, conforme o exemplo. \n\n

Exemplo: \n
Pergunta: "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Calcular juros>." \n
Resposta: \n
Código: EF09MA05
Descrição: Resolver e elaborar problemas que envolvam porcentagens, com a ideia de aplicação de percentuais sucessivos e a determinação das taxas percentuais, preferencialmente com o uso de tecnologias digitais, no contexto da educação financeira. \n\n

Contexto: {context} \n\n
Pergunta: {question}
"""
)

# Chain

In [37]:
chat_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt},
    return_source_documents=True
)

# Question

In [ ]:
# question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Excel> e o **Objetivo da Aula** é <Demostracao de graficos>."

# Answer

In [ ]:
# chat_chain.invoke({"query": question})

{'query': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Excel> e o **Objetivo da Aula** é <Demostracao de graficos>.',
 'result': 'Código: EF09MA06  \nDescrição: Utilizar diferentes representações gráficas para organizar e interpretar dados, incluindo gráficos de barras, gráficos de linhas e gráficos de setores, com o uso de tecnologias digitais.',
 'source_documents': []}